In [1]:
import tensorflow as tf
import numpy as np

In [2]:
data = np.loadtxt('../dataset/training_data_RNN.csv', delimiter=',', dtype='float32')

In [3]:
n_input = 36
n_step = 1500
n_hidden = 128
n_class = 11
n_batch = int(data.shape[0] / n_step)

learning_rate = 0.001
total_epoch = 20

In [4]:
x_data = data[:, :n_input]
x_data = x_data.reshape(n_batch, n_step, n_input)
y_data_temp = data[:, -1]

sess = tf.Session()
y_data_temp = np.subtract(y_data_temp, np.ones(y_data_temp.shape))
y_data_temp = tf.one_hot(y_data_temp, depth=n_class).eval(session=sess)

y_data = np.zeros([n_batch, n_class])
for i in range(n_batch):
    y_data[i] = y_data_temp[1500 * i]

print(x_data)
print(y_data)
print(x_data.shape)
print(y_data.shape)

[[[-1.005000e+00  2.290000e-01 -8.300000e-02 ...  3.159150e+02
    2.108154e+03  4.500000e+01]
  [-1.005000e+00  2.290000e-01 -8.300000e-02 ...  3.159150e+02
    2.108154e+03  4.500000e+01]
  [-1.005000e+00  2.290000e-01 -8.300000e-02 ...  3.159150e+02
    2.108154e+03  4.500000e+01]
  ...
  [-1.800000e-01 -1.008000e+00  5.900000e-02 ...  4.880000e-01
    2.538408e+03  1.180000e+02]
  [-1.800000e-01 -1.008000e+00  5.900000e-02 ...  3.050000e-01
    2.538408e+03  1.600000e+02]
  [-1.800000e-01 -1.008000e+00  5.900000e-02 ...  3.050000e-01
    2.538408e+03  1.600000e+02]]

 [[-1.017000e+00  1.280000e-01 -1.040000e-01 ...  4.207000e+00
    1.945644e+03  1.200000e+02]
  [-1.017000e+00  1.280000e-01 -1.040000e-01 ...  4.207000e+00
    1.945644e+03  1.200000e+02]
  [-1.023000e+00  1.080000e-01 -8.600000e-02 ...  1.274400e+01
    1.945644e+03 -1.610000e+02]
  ...
  [-1.970000e-01 -9.860000e-01  7.900000e-02 ...  6.100000e-02
    5.676500e+01  4.000000e+01]
  [-1.970000e-01 -9.860000e-01  7.90

In [5]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# BasicRNNCell, BasicLSTMCell, GRUCell
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

tf.summary.scalar('cost', cost)
tf.summary.scalar('accuracy', accuracy)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


<tf.Tensor 'accuracy:0' shape=() dtype=string>

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

for epoch in range(total_epoch):
    total_cost = 0

    _, cost_val = sess.run([optimizer, cost], feed_dict={X: x_data, Y: y_data})
    total_cost += cost_val

    print('Epoch:', '%04d' % (epoch + 1), 'Avg. cost =', '{:.3f}'.format(total_cost / n_batch))
    print('Accuracy =', sess.run(accuracy, feed_dict={X: x_data, Y: y_data}))
    
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=epoch)

Epoch: 0001 Avg. cost = 0.129
Accuracy = 0.15671642
Epoch: 0002 Avg. cost = 0.122
Accuracy = 0.14925373
Epoch: 0003 Avg. cost = 0.116
Accuracy = 0.15671642
Epoch: 0004 Avg. cost = 0.111
Accuracy = 0.1641791
Epoch: 0005 Avg. cost = 0.106
Accuracy = 0.1641791
Epoch: 0006 Avg. cost = 0.103
Accuracy = 0.14925373
Epoch: 0007 Avg. cost = 0.100
Accuracy = 0.1641791
Epoch: 0008 Avg. cost = 0.096
Accuracy = 0.1641791
Epoch: 0009 Avg. cost = 0.093
Accuracy = 0.17910448
Epoch: 0010 Avg. cost = 0.090
Accuracy = 0.18656716
Epoch: 0011 Avg. cost = 0.088
Accuracy = 0.18656716
Epoch: 0012 Avg. cost = 0.086
Accuracy = 0.18656716
Epoch: 0013 Avg. cost = 0.083
Accuracy = 0.19402985
Epoch: 0014 Avg. cost = 0.081
Accuracy = 0.19402985
Epoch: 0015 Avg. cost = 0.079
Accuracy = 0.18656716
Epoch: 0016 Avg. cost = 0.078
Accuracy = 0.18656716
Epoch: 0017 Avg. cost = 0.076
Accuracy = 0.19402985
Epoch: 0018 Avg. cost = 0.075
Accuracy = 0.20149253
Epoch: 0019 Avg. cost = 0.072
Accuracy = 0.20895523
Epoch: 0020 Avg.

In [8]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

print('Accuracy =', sess.run(accuracy, feed_dict={X: x_data, Y: y_data}))

Accuracy = 0.20895523
